In [1]:
# Turn off warnings 
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import seaborn as sns
sns.set_palette("Paired", 9)

In [5]:
## loading data
df = pd.read_csv('data/text_df.csv', index_col='Unnamed: 0')
df.head()

,Review Text,Rating,Recommended
0,Absolutely wonderful - silky and sexy and comf...,4,1
1,Love this dress! it's sooo pretty. i happene...,5,1
2,I had such high hopes for this dress and reall...,3,0
3,"I love, love, love this jumpsuit. it's fun, fl...",5,1
4,This shirt is very flattering to all due to th...,5,1


In [8]:
pip install -U textblob

     |████████████████████████████████| 636 kB 6.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [9]:
from textblob import TextBlob

In [10]:
testimonial= TextBlob("Textblob is amazingly simple to use. What great fun!")
testimonial.sentiment

Sentiment(polarity=0.39166666666666666, subjectivity=0.4357142857142857)

In [31]:
testimonial= TextBlob("This dress is good quality but the designers today really need to design their clothing line with the busty woman in mind")
testimonial.sentiment

Sentiment(polarity=0.44999999999999996, subjectivity=0.4)

In [35]:
df['polarity'] = df['Review Text'].apply(lambda x: TextBlob(x).sentiment.polarity)
df

,Review Text,Rating,Recommended,polarity
0,Absolutely wonderful - silky and sexy and comf...,4,1,0.633333
1,Love this dress! it's sooo pretty. i happene...,5,1,0.339583
2,I had such high hopes for this dress and reall...,3,0,0.073675
3,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0.550000
4,This shirt is very flattering to all due to th...,5,1,0.512891
...,...,...,...,...
23481,I was very happy to snag this dress at such a ...,5,1,0.552667
23482,"It reminds me of maternity clothes. soft, stre...",3,1,0.091667
23483,"This fit well, but the top was very see throug...",3,0,0.414286
23484,I bought this dress for a wedding i have this ...,3,1,0.322222


In [38]:
df.loc[23481,'Review Text']

"I was very happy to snag this dress at such a great price! it's very easy to slip on and has a very flattering cut and color combo."